# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. SQL Agent 应用开发

## 2.1 简介

和向量数据库一样，假如我们想把关系数据库转变为智能体的工具，第一步就是要获取到一个存储着和任务相关内容的关系数据库。

这里我们可以从 google 那下载一个经典的关系数据库 Chinook.db 进行演示。

这是一个模拟数字音乐商店（Digital Media Store）的数据库。它包含了音乐专辑、艺术家、客户、发票、员工等数据。也推荐大家使用自己的数据库进行尝试。

## 2.2 应用开发

我们可以先使用 requests 库下载这个库并存放到本地的文件夹中：

In [3]:
import requests, pathlib

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
local_path = pathlib.Path("Chinook.db")

if local_path.exists():
    print(f"{local_path} already exists, skipping download.")
else:
    response = requests.get(url)
    if response.status_code == 200:
        local_path.write_bytes(response.content)
        print(f"File downloaded and saved as {local_path}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Chinook.db


下载完成后，我们可以使用 LangChain 中的 SQLDatabase 来实现连接。这里我们打印一下数据库语言、可用的表名称以及通过 db.run 执行了一个简单的 SQL 查询语句：

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


完成数据准备后，我们就可以开始尝试与关系数据库进行交互了。对于这类结构化数据，通常需要通过 SQL 语句来完成查询与分析操作。然而，手动编写 SQL 查询既繁琐又容易出错，特别是在面对复杂表结构或多表关联时。

为此，LangChain 提供了一个专门面向关系数据库的工具包 — SQLDatabaseToolkit。它能够结合大语言模型的自然语言理解能力，自动将用户提出的问题转化为正确的 SQL 查询语句并执行，从而显著简化数据库交互过程。我们只需将语言模型（LLM）和数据库实例（DB）传入该工具包即可完成初始化，整个查询过程无需手动编写 SQL：

In [5]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.chat_models import ChatTongyi

model = ChatTongyi(model="qwen-max")
toolkit = SQLDatabaseToolkit(db=db, llm=model)

我们可以打印一下这里面包含的工具信息及对应的描述：

In [6]:
tools = toolkit.get_tools()
for tool in tools:
  print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



下一步我们就可以准备开始组装 agent 了，和之前一样，假如我们不需要考虑记忆的话，我们需要准备模型、工具以及系统提示词三部分。

前两部分我们已经准备好了，下面我们就来看看如何设计系统提示词：

In [7]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)

然后我们就可以将这三部分内容串联起来组合成最后的 agent：

In [8]:
from langchain.agents import create_agent

agent = create_agent(model, tools, system_prompt=system_prompt)

然后将这个 agent 进行调用：

In [9]:
question = "Which genre on average has the longest tracks?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_17de76bc7e3a4358bb0f9c)
 Call ID: call_17de76bc7e3a4358bb0f9c
  Args:
    tool_input: 
  sql_db_schema (call_12f7cfaed7544a88ad9dd1)
 Call ID: call_12f7cfaed7544a88ad9dd1
  Args:
    table_names: tracks, genres
================================= Tool Message =================================
Name: sql_db_schema

Error: table_names {'tracks', 'genres'} not found in database
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_38f5b6d1a5a048888d927f)
 Call ID: call_38f5b6d1a5a048888d927f
  Args:
    table_names: Track, Genre
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Nam

其实对于让模型直接使用 SQL 语言进行操作时有风险的，虽然提示词已经进行了提醒，但是我们还可以给这个智能体加上人类介入的中间件 HumanInTheLoopMiddleware 来进一步规避最后使用 sql_db_query 工具时进行查询或调用的风险：

In [10]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

agent = create_agent(model, tools, system_prompt=system_prompt,
    middleware=[HumanInTheLoopMiddleware( 
            interrupt_on={"sql_db_query": True}, 
            description_prefix="Tool execution pending approval" )], 
    checkpointer=InMemorySaver(), 
)

然后我们同样可以对其进行调用，但这里添加了一个判定，假如遇到了 "__interrupt__" 的话就会停下来并返回相关的调用信息：

In [11]:
question = "Which genre on average has the longest tracks?"
config = {"configurable": {"thread_id": "1"}} 

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    config, stream_mode="values"):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step: 
        print("INTERRUPTED:") 
        interrupt = step["__interrupt__"][0] 
        for request in interrupt.value["action_requests"]: 
            print(request["description"]) 
    else:
        pass

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_bf20d9019f8044119e2ccc)
 Call ID: call_bf20d9019f8044119e2ccc
  Args:
    tool_input: 
  sql_db_schema (call_71fd9cdf0a144542b8877a)
 Call ID: call_71fd9cdf0a144542b8877a
  Args:
    table_names: tracks,genres
================================= Tool Message =================================
Name: sql_db_schema

Error: table_names {'tracks', 'genres'} not found in database
================================== Ai Message ==================================

It seems there was an error because the table names 'tracks' and 'genres' were not correctly recognized in the database. Let me first confirm the correct table names from the list of available tables.
Tool Calls:
  sql_db_schema (call_9fdfb8b710bc4a1c9ad89c)
 Call ID: call_9fdfb8b710bc4a1c9ad89

出现中断后我们可以通过 Command 里传入同意、不同意或修改几个可选项，然后让其进一步完成即可，最后模型找到足够的信息就会生成最终的回复：

In [12]:
from langgraph.types import Command 

for step in agent.stream(
    Command(resume={"decisions": [{"type": "approve"}]}), 
    config, stream_mode="values",):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]
        for request in interrupt.value["action_requests"]:
            print(request["description"])
    else:
        pass

================================== Ai Message ==================================

To find out which genre on average has the longest tracks, I will need to join the `Track` and `Genre` tables on the `GenreId` field. Then I can group by the genre and calculate the average track length (in milliseconds) for each genre. Finally, I'll order the results in descending order of the average length and limit the result to 1 to get the genre with the longest average track.

Let me now construct and execute the SQL query.
Tool Calls:
  sql_db_query (call_8485fe6487cc43248df0d4)
 Call ID: call_8485fe6487cc43248df0d4
  Args:
    query: SELECT g.Name AS GenreName, AVG(t.Milliseconds) AS AverageLength FROM Track t JOIN Genre g ON t.GenreId = g.GenreId GROUP BY g.Name ORDER BY AverageLength DESC LIMIT 1;
================================== Ai Message ==================================

To find out which genre on average has the longest tracks, I will need to join the `Track` and `Genre` tables on the `